In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
bureau = pd.read_csv(PATH + 'bureau.csv')
bb = pd.read_csv(PATH + 'bureau_balance.csv')
b = bureau.copy()
'done'
df = pd.concat([train[['SK_ID_CURR']], test[['SK_ID_CURR']]], axis = 0)
print('bshape',b.shape)
print('dfshaoe',df.shape)

bshape (1716428, 17)
dfshaoe (356255, 1)


## Functions

In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def multi_name(col1, col2): return col1 + '_times_' + col2
def multi(df, col1, col2): return df[col1] * df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

## Length of history
## Define _day minus days_credit, as the length of the credit, and construct weighted days_credit, then sum & merge

In [3]:
# Smallest among them is days credit
days_list = ['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT','DAYS_CREDIT_UPDATE']
days = ['CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT','DAYS_CREDIT_UPDATE']

for col in days:
    tmp = minus_name(col, 'DAYS_CREDIT')
    b[tmp] = minus(b, col, 'DAYS_CREDIT')
    print(tmp,'lengthof positive,', (b[tmp]>=0).sum()/len(b))
print('bshape',b.shape)
print('dfshaoe',df.shape)

CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT lengthof positive, 1.0
DAYS_CREDIT_ENDDATE_minus_DAYS_CREDIT lengthof positive, 0.9384087185713587
DAYS_ENDDATE_FACT_minus_DAYS_CREDIT lengthof positive, 0.6308187701435772
DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT lengthof positive, 0.9999213482884223
bshape (1716428, 21)
dfshaoe (356255, 1)


In [4]:
# Do a little fixing for two columns, so that the minus is always non-negative
col_fix = ['DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT','DAYS_CREDIT_ENDDATE_minus_DAYS_CREDIT','DAYS_ENDDATE_FACT_minus_DAYS_CREDIT','CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT']
for col in col_fix:
    b[col].fillna(0, inplace = True)
    x = b[col]<0
    b.loc[x] = 0
#     print( tmp,'neg',(b[tmp]<0).sum())
print('bshape',b.shape)
print('dfshaoe',df.shape)

bshape (1716428, 21)
dfshaoe (356255, 1)


In [5]:
# import numpy as np
# tmp = 1 + np.sqrt(  abs(b['DAYS_CREDIT']) / 365 )
# b['WGT_DAYS_CREDIT'] = 1/( 1 + np.sqrt(  abs(b['DAYS_CREDIT']) / 365 ) )

# b['WGT_credit_length1'] = b['WGT_DAYS_CREDIT'] * b['CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT']
# b['WGT_credit_length2'] = b['WGT_DAYS_CREDIT'] * b['DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT']
# print('bshape',b.shape)
# print('dfshaoe',df.shape)

# Some other days substraction

In [6]:
days_list = ['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT','DAYS_CREDIT_UPDATE']

def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

b[minus_name('CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_UPDATE')] = minus(b, 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_UPDATE')
# b[minus_name('CREDIT_DAY_OVERDUE', 'DAYS_ENDDATE_FACT')] =  minus(b, 'CREDIT_DAY_OVERDUE', 'DAYS_ENDDATE_FACT')
b[minus_name('DAYS_CREDIT_UPDATE', 'DAYS_CREDIT_ENDDATE')] =  minus(b, 'DAYS_CREDIT_UPDATE', 'DAYS_CREDIT_ENDDATE')
# b[minus_name('DAYS_CREDIT_UPDATE', 'DAYS_ENDDATE_FACT')] =  minus(b, 'DAYS_CREDIT_UPDATE', 'DAYS_ENDDATE_FACT')
print('bshape',b.shape)
print('dfshaoe',df.shape)

'''
AMT_CREDIT_SUM_DEBT : WGT_DAYS_CREDIT ,DAYS_ENDDATE_FACT
AMT_CREDIT_SUM      : DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
                    : DAYS_ENDDATE_FACT_minus_DAYS_CREDIT


'''
top = ['AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM','AMT_CREDIT_SUM_LIMIT']
bot = [ 'DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT_minus_DAYS_CREDIT' ]
for i in top:
    for j in bot:
        print(ratio_name(i,j))
        b[ratio_name(i,j)] = ratio(b, i, j)
        
print(b.shape)

bshape (1716428, 23)
dfshaoe (356255, 1)
AMT_CREDIT_SUM_DEBT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM_DEBT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_LIMIT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM_LIMIT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
(1716428, 29)


#  credit types:

In [7]:
# Types of credits: credit-type, credit_currency
# For a certain id_curr, do nunique, count, one hot encoding(then count)
gp_col = 'SK_ID_CURR'
col = ['CREDIT_TYPE','CREDIT_CURRENCY', 'CREDIT_ACTIVE']
for x in col:
    print(x)
    group1 = b[[gp_col, x]].groupby(gp_col)[[x]].nunique().reset_index().rename(index = str, columns = {x:'unique_'+x})
    col = 'SK_ID_CURR'
    df = df.merge(group1, on  = gp_col, how = 'left')

CREDIT_TYPE
CREDIT_CURRENCY
CREDIT_ACTIVE


# Besides ratios, and numerical, final processing: haoyan's version

# Others

In [9]:
'''
AMT_CREDIT_SUM_DEBT : WGT_DAYS_CREDIT ,DAYS_ENDDATE_FACT
AMT_CREDIT_SUM      : DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
                    : DAYS_ENDDATE_FACT_minus_DAYS_CREDIT

'''
top = ['AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM','AMT_CREDIT_SUM_LIMIT']
bot = [ 'DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT_minus_DAYS_CREDIT' ]
for i in top:
    for j in bot:
        print(ratio_name(i,j))
        b[ratio_name(i,j)] = ratio(b, i, j)
        
print(b.shape)

AMT_CREDIT_SUM_DEBT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM_DEBT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_LIMIT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM_LIMIT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
(1716428, 29)


In [10]:
ignored = ['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']
set_numerical = set(b.columns) - set(ignored)



{'AMT_ANNUITY',
 'AMT_CREDIT_MAX_OVERDUE',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_DEBT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_DEBT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_LIMIT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_LIMIT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_SUM_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT',
 'CNT_CREDIT_PROLONG',
 'CREDIT_DAY_OVERDUE',
 'CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT_UPDATE',
 'DAYS_CREDIT',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_CREDIT_ENDDATE_minus_DAYS_CREDIT',
 'DAYS_CREDIT_UPDATE',
 'DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT',
 'DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'DAYS_ENDDATE_FACT_minus_DAYS_CREDIT'}

In [12]:
bureau_copy = pd.read_csv(PATH + 'bureau.csv')
set_numerical = [x for x in set_numerical if x not in bureau_copy.columns]
set_numerical

['AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT',
 'AMT_CREDIT_SUM_DEBT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT',
 'DAYS_CREDIT_ENDDATE_minus_DAYS_CREDIT',
 'AMT_CREDIT_SUM_LIMIT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT',
 'AMT_CREDIT_SUM_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT_minus_DAYS_CREDIT',
 'AMT_CREDIT_SUM_DEBT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT',
 'AMT_CREDIT_SUM_LIMIT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE',
 'CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT_UPDATE',
 'DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT']

In [13]:
# numerical
for x in set_numerical:
    print(x)
    df = df.merge(numerical(b, gp_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=gp_col, how='left')
    

columns = []
for x in df.columns:
    tmp = 'bureau_' + x if x != gp_col else x
    columns.append(tmp)
df.columns = columns
'done'
print('bshape',b.shape)
print('dfshaoe',df.shape)

AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_DEBT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT
DAYS_CREDIT_ENDDATE_minus_DAYS_CREDIT
AMT_CREDIT_SUM_LIMIT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_DEBT_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT
AMT_CREDIT_SUM_LIMIT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE
CREDIT_DAY_OVERDUE_minus_DAYS_CREDIT_UPDATE
DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT
bshape (1716428, 29)
dfshaoe (356255, 64)


## merge

In [ ]:
print(df.columns)
df.shape

In [14]:
df.to_pickle(PATH + 'inter/bureau_sup.pkl')
'done'

'done'

In [5]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'
df = pd.read_pickle(PATH + 'inter/bureau_sup.pkl')
bureau = pd.read_pickle(PATH + 'inter/bureau2curr.pkl')

In [6]:
bureau.shape

(356255, 554)

In [7]:
df = df.merge(bureau, on = 'SK_ID_CURR', how = 'left')

In [8]:
df.head()

,SK_ID_CURR,bureau_unique_CREDIT_TYPE,bureau_unique_CREDIT_CURRENCY,bureau_unique_CREDIT_ACTIVE,bureau_AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT_mean,bureau_AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT_median,bureau_AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT_max,bureau_AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT_min,bureau_AMT_CREDIT_SUM_divide_DAYS_ENDDATE_FACT_minus_DAYS_CREDIT_sum,bureau_AMT_CREDIT_SUM_DEBT_divide_DAYS_CREDIT_UPDATE_minus_DAYS_CREDIT_ENDDATE_mean,...,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_max,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_min,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_sum,bureau_AMT_CREDIT_MAX_OVERDUE_divide_DAYS_CREDIT_UPDATE_std,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_mean,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_median,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_max,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_min,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_sum,bureau_STATUS_NUM_divide_MONTHS_BALANCE_sum_std
0,100002,2.0,1.0,2.0,60518.786064,498.395010,450000.000000,0.000000,484150.288512,-104.232824,...,-0.0,-152.837727,-158.26821,67.760926,-0.135012,-0.136674,0.0,-0.258263,-1.080096,0.110696
1,100003,2.0,1.0,2.0,202655.016924,285.639111,810000.000000,48.789474,810620.067696,0.000000,...,-0.0,-0.000000,0.00000,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN
2,100004,1.0,1.0,1.0,1824.069565,1824.069565,3501.400000,146.739130,3648.139130,0.000000,...,-0.0,-0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3,100006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,1.0,1.0,1.0,398.501362,398.501362,398.501362,398.501362,398.501362,0.000000,...,-0.0,-0.000000,0.00000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


In [9]:
df.to_pickle(PATH+'test/new_bureau.pkl')